In [2]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

In [4]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [5]:
conn.rollback()
import pandas as pd
from datetime import datetime

# 월별 범위 생성
periods = []
for year in range(2015, 2025):
    for month in range(1, 13):
        # 다음 달의 첫날을 계산
        if month == 12:
            next_month_start = f'{year+1}-01-01'
        else:
            next_month_start = f'{year}-{month+1:02d}-01'
            
        periods.append({
            'name': f'{year}-{month:02d}',  # 2015-01 형식
            'start': f'{year}-{month:02d}-01',
            'end': next_month_start
        })

# 결과를 저장할 리스트
all_results = []

# 각 기간별 데이터 조회
for period in periods:
    qry = """
        SELECT 
            COUNT(ch."createdAt") as total_count,
            SUM(CASE WHEN ch."status" = 'lost' THEN 1 ELSE 0 END) as lost_count
        FROM credit_history ch
        WHERE ch."openedAt" >= %s 
        AND ch."openedAt" < %s
        """
    cursor.execute(qry, (period['start'], period['end']))
    result = cursor.fetchone()
    
    # 결과를 리스트에 추가
    all_results.append({
        '기간': period['name'],
        '총 대출 수': result[0],
        '부실 대출 수': result[1]
    })

# DataFrame 생성 및 CSV 저장
df = pd.DataFrame(all_results, columns=['기간', '총 대출 수', '부실 대출 수'])
df.to_csv('./월별_대출현황.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

        기간  총 대출 수  부실 대출 수
0  2015-01       0      NaN
1  2015-02       1      0.0
2  2015-03       0      NaN
3  2015-04       0      NaN
4  2015-05       2      1.0
